# Currency Conversion Tool

In [95]:
# 1st tool
from langchain_core.tools import tool
import requests
from typing import Annotated
from langchain_core.tools import InjectedToolArg
from dotenv import load_dotenv
load_dotenv()

@tool
def get_conversion_factor(base_currency:str,target_currency:str)->float:
    """this tool fetch the currency conversion factor between a given base currency and a target currency"""
    url=f"https://v6.exchangerate-api.com/v6/0ac5c1dd956231a4da75bd12/pair/{base_currency}/{target_currency}"
    response=requests.get(url)
    return response.json()

# 2nd tool

@tool
def convert_currency(base_currency:int,conversion_factor:Annotated[float,InjectedToolArg])-> float:
    "This tool convert the base currenty into target currency"
    return base_currency * conversion_factor


In [96]:
check_conversion_factor=get_conversion_factor.invoke({"base_currency": "USD","target_currency": "PKR"})

In [97]:
check_conversion_factor

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1753660801,
 'time_last_update_utc': 'Mon, 28 Jul 2025 00:00:01 +0000',
 'time_next_update_unix': 1753747201,
 'time_next_update_utc': 'Tue, 29 Jul 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'PKR',
 'conversion_rate': 283.3984}

In [98]:
convert_currency.invoke({'base_currency':100,'conversion_factor':283})

28300.0

In [99]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI()

In [100]:
# bind tools
llm_tool=llm.bind_tools([get_conversion_factor,convert_currency])

In [101]:
llm_tool

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000016927341BA0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000016927343E10>, root_client=<openai.OpenAI object at 0x00000169236FDBD0>, root_async_client=<openai.AsyncOpenAI object at 0x00000169272FAFD0>, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_conversion_factor', 'description': 'this tool fetch the currency conversion factor between a given base currency and a target currency', 'parameters': {'properties': {'base_currency': {'type': 'string'}, 'target_currency': {'type': 'string'}}, 'required': ['base_currency', 'target_currency'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'convert_currency', 'description': 'This tool convert the base currenty into target currency', 'parameters': {'properties': {'base_currency': {'type': 'int

In [102]:
from langchain_core.messages import HumanMessage


messages=[HumanMessage(content="What is the conversion factor between USD and PKR? and also convert the amount of 100US into PKr")]

In [103]:
ai_msg=llm_tool.invoke(messages)

In [104]:
messages.append(ai_msg)

In [105]:
messages

[HumanMessage(content='What is the conversion factor between USD and PKR? and also convert the amount of 100US into PKr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_K9OoZ0bAZlvRTV96LzvaSDfk', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "PKR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_hZ9XtZgyMHwnbIOrjEm3VbgO', 'function': {'arguments': '{"base_currency": 100}', 'name': 'convert_currency'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 112, 'total_tokens': 164, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-ByJtidmd8xym8AGPqUce5BcXMCauC', 'finish_reason': 't

In [106]:
# tool execution

ai_msg.tool_calls


[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'PKR'},
  'id': 'call_K9OoZ0bAZlvRTV96LzvaSDfk',
  'type': 'tool_call'},
 {'name': 'convert_currency',
  'args': {'base_currency': 100},
  'id': 'call_hZ9XtZgyMHwnbIOrjEm3VbgO',
  'type': 'tool_call'}]

In [107]:
import json
for tool_call in ai_msg.tool_calls:
    if tool_call['name'] =='get_conversion_factor':
        tool_msg1=get_conversion_factor.invoke(tool_call)
        conversion_factor=json.loads(tool_msg1.content)['conversion_rate']
        messages.append(tool_msg1)
    elif tool_call['name'] =='convert_currency':
        tool_call['args']['conversion_factor']=conversion_factor
        tool_msg2=convert_currency.invoke(tool_call)
        messages.append(tool_msg2)

In [108]:
messages

[HumanMessage(content='What is the conversion factor between USD and PKR? and also convert the amount of 100US into PKr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_K9OoZ0bAZlvRTV96LzvaSDfk', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "PKR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_hZ9XtZgyMHwnbIOrjEm3VbgO', 'function': {'arguments': '{"base_currency": 100}', 'name': 'convert_currency'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 112, 'total_tokens': 164, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-ByJtidmd8xym8AGPqUce5BcXMCauC', 'finish_reason': 't

In [109]:
result=llm_tool.invoke(messages)

In [110]:
result.content

'The conversion factor between USD and PKR is 283.3984.\n\nWhen 100 USD is converted to PKR, it amounts to 28,339.84 PKR.'